# Evaluate Kahoot performance during a course

## Evaluate points

In [29]:
import os
import pandas
import numpy as np

In [30]:
# Initialize same random state every time
random_state = np.random.RandomState(20210104)

### Read data from Excell files

In [31]:
directory_name = "A:\\1docs\\aktualis\\2020-2021-1 Oktatas\\Korszeru_nuklearis_energiatermeles\\kahoot"
sheet_name = "Final Scores"
list_of_all_files = os.listdir(directory_name)
list_of_all_files

['Korszerű fúzió 1-1..xlsx',
 'Korszerű fúzió 3-3.xlsx',
 'Korszerű fúzió 3-1..xlsx',
 'Korszerű fúzió 3-2.xlsx',
 'Korszerű fúzió 2-3..xlsx',
 'Korszerű fúzió 1-1_2..xlsx',
 'Korszerű fúzió 2-2..xlsx',
 'Korszerű fúzió 4-1.xlsx',
 'Korszerű fúzió 5-1.xlsx',
 'Korszerű fúzió 4-2.xlsx',
 'Korszerű nukleáris energiatermelés 2..xlsx',
 'Korszerű nukleáris energiatermelés 1..xlsx',
 'KoNET 7. előadás - AE építés.xlsx',
 'KoNET 6. előadás - PWRs _ #2.xlsx',
 'KoNET 6. előadás - PWRs _ #1.xlsx']

In [32]:
# list_of_files = [elem for elem in list_of_all_files if elem.find("kahoot-")==0]
list_of_files = list_of_all_files

### Players corrections

In [33]:
def convert_players(old_player):
    if old_player == "":
        return ""
    old_player = str(old_player).upper()
    return old_player

conv_players=np.vectorize(convert_players)

### Read data from Excell files and perform transformations

In [34]:
scores = pandas.DataFrame()
for file_name in list_of_files:
    scores_single = pandas.read_excel(directory_name+"\\"+file_name, sheet_name=sheet_name, header=1, 
                                      skiprows=[1], skipfooter=1,
                                      converters={"Player":convert_players})
    # Give points for perfect tests
    scores_single["Points"] = scores_single["Incorrect Answers"]==0 
    # Give points to two further tests randomly selected with Kahoot points acting as weights 
    weights = scores_single[scores_single["Points"]==False]["Total Score (points)"].values
    weights = weights / sum(weights)
    selected_ind = random_state.choice(scores_single[scores_single["Points"]==False].index,2,p=weights)
    scores_single.iloc[selected_ind,scores_single.columns.get_loc("Points")] = True
    # Append to summary table
    scores = scores.append(scores_single)

In [35]:
scores[scores["Points"]]

,Rank,Player,Total Score (points),Correct Answers,Incorrect Answers,Unnamed: 5,Points
0,1,ITDDXR,5806,9,0,NaN,True
2,3,VU63MO,5215,9,0,NaN,True
18,19,OLLW2U,3669,7,2,NaN,True
32,33,WS0J9W,2449,5,4,NaN,True
0,1,GEU1LX,4964,6,0,NaN,True
...,...,...,...,...,...,...,...
2,3,GYDEZC,2373,3,0,NaN,True
3,4,JN7OKQ,2360,3,0,NaN,True
4,5,I42F17,1982,3,0,NaN,True
14,15,IU07OE,1165,2,1,NaN,True


### Aggregate results of individual Players

In [36]:
scores_aggregate = scores.groupby('Player',as_index=False).agg({'Rank': ['mean', 'count'],
                                                                'Total Score (points)': 'sum',
                                                                'Correct Answers': 'sum',
                                                                'Incorrect Answers': 'sum',
                                                                'Points': 'sum'})

In [37]:
scores.columns

Index(['Rank', 'Player', 'Total Score (points)', 'Correct Answers',
       'Incorrect Answers', 'Unnamed: 5', 'Points'],
      dtype='object')

In [38]:
scores_aggregate.columns=[ 'Players', 'Rank', 'Count', 'Total Score (points)', 'Correct Answers', 
                          'Incorrect Answers', 'Points']


In [39]:
scores_aggregate

,Players,Rank,Count,Total Score (points),Correct Answers,Incorrect Answers,Points
0,AG4JUQ,23.428571,7,19680,31,22,0.0
1,AG4JUQ KIDOBOT,31.000000,1,1926,3,4,0.0
2,BCUCD3,11.500000,12,45137,61,15,3.0
3,BTEYES,12.733333,15,51550,70,23,4.0
4,C4E2B5,21.000000,1,3599,7,2,0.0
5,C9VAQE,17.000000,2,4841,9,3,0.0
6,CHAZX6,13.250000,8,33407,45,12,1.0
7,EEO8MF,14.500000,10,37397,51,14,1.0
8,EEO8MF L,20.000000,1,2334,3,2,0.0
9,FA1PJ3,13.571429,7,30283,41,7,2.0


### Print result

In [40]:
writer = pandas.ExcelWriter(directory_name+"\\"+'kahoot_results.xlsx')
scores_aggregate.to_excel(writer,'Results')
writer.save()

## Evaluate question statistics